In [1]:
# Importowanie bibliotek
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [2]:
# Ustawienie ziarna losowości
np.random.seed(42)

In [3]:
# Wczytanie danych (zbior odręcznych cyfr z sklearn)
data = datasets.load_digits()
images = data['images']
targets = data['target']

In [4]:
# Skalowanie pikseli (standaryzacja do [0, 1])
images = images / 16.0

In [5]:
# Podział danych na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(images, targets, test_size=0.25, random_state=42)

In [6]:
# Spłaszczenie obrazów (wektory 64-elementowe)
X_train = X_train.reshape(-1, 64)
X_test = X_test.reshape(-1, 64)

In [7]:
# Ustawienie parametrów kwantowego systemu
n_qubits = 10  # Liczba kubitów odpowiada liczbie klas
dev = qml.device("default.qubit", wires=n_qubits)

In [8]:
# Funkcja kodowania danych w przestrzeni kwantowej
def encode_data(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

In [9]:
# Złożony kwantowy ansatz (obwód parametryczny)
def variational_circuit(weights):
    for layer in weights:
        for i in range(n_qubits):
            qml.RX(layer[i], wires=i)
            qml.RZ(layer[i + n_qubits], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])

In [10]:
# Kwantowy obwód złożony z kodowania i ansatza
@qml.qnode(dev)
def quantum_circuit(params, x):
    encode_data(x)
    variational_circuit(params)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [11]:
# Funkcja kosztu oparta na entropii krzyżowej
def cost(params, X, y):
    predictions = np.array([quantum_circuit(params, x) for x in X])
    softmax_preds = np.exp(predictions) / np.sum(np.exp(predictions), axis=1, keepdims=True)
    one_hot_y = np.eye(num_classes)[y]
    return -np.mean(np.sum(one_hot_y * np.log(softmax_preds + 1e-10), axis=1))

In [12]:
# Trenowanie modelu dla wszystkich klas
num_classes = 10
opt = qml.GradientDescentOptimizer(stepsize=0.1)
num_epochs = 30

In [ ]:
# Parametry ansatza (więcej warstw)
params = np.random.uniform(low=-np.pi, high=np.pi, size=(num_epochs, 2 * n_qubits))
losses = []

for epoch in range(num_epochs):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)
    current_loss = cost(params, X_train, y_train)
    losses.append(current_loss)
    print(f"Epoch {epoch+1}: Loss = {current_loss:.4f}")

In [ ]:
# Predykcja dla danych testowych
def predict(X):
    predictions = np.array([quantum_circuit(params, x) for x in X])
    return np.argmax(predictions, axis=1)


In [ ]:
# Ewaluacja modelu
y_pred = predict(X_test)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

In [ ]:
# Macierz konfuzji
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
